# Example workflow of the PAWN global sensitivity analysis method using PRMS-Python on three arbitrary PRMS parameters 

The PAWN global, moment-independent, sensitivity analysis (SA) method is relatively straightforward to implement for any PRMS parameter using PRMS-Python objects. This example template can be modified to create the data necessary (emprical CDFs) for PAWN SA on any number of physical PRMS paraemters for any PRMS model. More information on the PAWN method can be found in the manuscript [here](http://dx.doi.org/10.1016/j.envsoft.2015.01.004). A case study using a slightly modified version of this script was used to conduct PAWN SA on 8 parameters that comprise the degree day solar radiation method in PRMS, details can be found in the PRMS-Python manuscript [here](). The control parameters and variable names for the experimental setup of PAWN as stated the literature are used within this code template, similarly this example code is heavily commented for clarity.

In [2]:
import numpy as np
import pandas as pd
import os, json
from prms_python import Data, Optimizer, Parameters, util
from prms_python.optimizer import resample_param as resample
from prms_python.optimizer import OptimizationResult
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

## Define paths to initial model inputs, measured data, and initialize input objects

In [3]:
param_path 'path/to/model/parameters'
data = Data('path/to/model/data') # this example does not modify climate forcing so a single instance is fine
control_path = 'path/to/model/control'
work_directory = '/path/to/output/directory'
measrd_path = 'path/to/measured/output.csv'
PRMS_output_variable = 'name_of_PRMS_output_variable_for_SA' # e.g. "basin runoff 1" or Y in PAWN

Note, to avoid platform specific path errors use os.path.join() as opposed to strings with platform path separators. For example if the path to the control file is $HOME/prms/control then use:

```python
home = os.path.expanduser("~")
control_path = os.path.join(home, 'prms', 'control')
```


## Define sampling method, optimization title, and experimental setup control parameters

In [11]:
resample_meth = 'uniform' # sampling parameter values from a uniform distribution is one way for global SA
title = 'ddsolrad_PAWN' # title optional
nprocs = 8 # number of physical or logical processing cores to use

# PAWN related experimental setup variables
M = 3 # number of input factors for SA (parameters in this case)
Nuc = 4000 # number of simulations to build the unconditional CDF
Nc = 20 # number of times to resample each conditioning parameter
n = 100 # number of simulations for buidling each conditional CDF

# stages are used by the Optimizer object, 
# in PAWN these are essentially the names of each input factor 1,2,...,M
stage_names = ['unconditional', 'cond_p1', 'cond_p2', 'cond_p3'] 
# each conditional stage is the exclusion of 1 of the 3 input parameters which is held 'constant' Nc times
param_names_for_each_stage = [
                                ['p1_name', 'p2_name', 'p3_name'], 
                                ['p2_name', 'p3_name'], 
                                ['p1_name', 'p3_name'], 
                                ['p1_name', 'p2_name']
                             ]

## Conduct Nuc simulations on all M parameters to build the unconditional CDF

In [ ]:
stage_name = stage_names[0] # "unconditional"
archive_dir = os.path.join(work_directory,"{}_archived".format(stage_name))
# make archive directory if it doesn't exist
if not os.path.isdir(archive_dir): 
    os.mkdir(archive_dir)
# create an Optimizer instance and call monte_carlo method
optr = Optimizer(Parameters(param_path), data, control_path, work_directory, title=title)
optr.monte_carlo(
                 measured_path, 
                 param_names_for_each_stage[0], 
                 PRMS_output_variable, 
                 method=resample_meth,
                 n_sims=Nuc, 
                 nproc=nprocs, 
                 stage=stage_name
                 )
# optionally archive output to reduce disk space
result = OptimizationResult(work_directory, stage=stage_name)
result.archive()

## Create conditional CDFs Nc times for each parameter, storing conditioning parameter values

In [ ]:
for nc in range(Nc): # number of conditional param bootstrap resamples- distributable to multiple machines/nodes
    for i, stage in enumerate(stage_names): # xi param of M total params
        if stage=='unconditional': # already built unconditional CDF- skip
            continue
        # use unconditional param list set minus conditioning param list to find conditioning param name
        conditional_param = list(set(stage_names['unconditional']) - set(stage_names[stage]))[0]
        # make a Parameter instance to resample the conditioning param and others 
        params = Parameters(param_path)
        xi_values = resample(params, conditional_param)
        params[conditional_param] = xi_values # assign resampled conditioning parameter (xi) to Parameter object
        stage_name = stage+str(nc) # add nc for nc'th bootstrap resampling round
        # create output archive directory to hold info on xi conditioning values and correspinding model output
        archive_dir = os.path.join(work_directory,"{}_archived".format(stage_name))
        if not os.path.isdir(archive_dir):
            os.mkdir(archive_dir)
        if xi_values.shape == (): # some parameters may be single valued, this worked for me
            with open(os.path.join(archive_dir,'{}_Nc_{}.txt'.format(conditional_param,i)), 'w') as outf:
                outf.write(str(xi_values))
        else: # ndarrays can be dumped to a text file using numpy
            np.savetxt(os.path.join(archive_dir,'{}_Nc_{}.txt'.format(conditional_param,i)), xi_values, fmt="%f")
        # create an Optimizer instance and call monte_carlo method with the correct parameter modifications
        optr = Optimizer(params, data, control_path, work_directory, title=title)
        optr.monte_carlo(
                         measured_path, 
                         param_names_for_each_stage[i], 
                         PRMS_output_variable, 
                         method=resample_meth,
                         n_sims=n,  
                         nproc=nprocs, 
                         stage=stage_name
                         )
        # optionally archive output to reduce disk space
        result = OptimizationResult(work_directory, stage=stage_name)
        result.archive()

## That's it, now analyze output to calculate sensitivity indices for each input parameter

Although this example does not include analysis of results, it is straightforward to build output CDFs from the archived JSON files. For example of accessing output from these files please refer to the Jupyter Notebooks that packs with PRMS-Python for the `OptimizationResult` object. For convenience we added Python functions that calculate emprical CDFs and the Kolmogorov-Smirnov distance between two CDFs in the `prms_python.util` module. These functions can be used to easily calculate the PAWN sensitivity analysis from the results produced using the template above.  